# Evaluation of Classifiers on Data Other than the Bioreactor Samples

This notebook analyzes the accuracy of the SVM classifier on data other than the bioreactor samples.
The analysis makes use of data built by ClassificationData in the tools directory.
This class augments the global namespace to include data used in analysis.

The training data are:
* all genes (not just regulators);
* averaged;
* do not include T1 (since only one observtion)

# TO DO
1. Plot progression for individual samples for Bioreactor
1. Evaluate average of replications for other progressions
1. Examine the contributions to the classification by gene in the samples that don't result in mycobactin being a strong predictor.
1. Do cross correlation studies of genes to check ortogonality.
1. **Check calculation of pooled, whether average log**
1. Structure codes for the analyses conducted. This structure should be reflected in ``DATA_DCT``.
   1. Selection of genes: myco, myco +, all genes
   1. How counts are processed
      1. Reference - T0/biostat, T0/data source, pooled/data source
1. Evaluation of classifiers
   1. Performance on biostat data
   1. Performance on other samples
      1. Time consistency
      1. Replication consistency
      1. Consistency with expectation
1. Analyses for Eliza:
   1. Accuracy with all 11 and the accuracy as you add in more genes (same as what you just sent but with all 11).
   1. Could you test with just the 10 mycobactin genes (Rv2377c-Rv2386c)?
   1. Could you test with just bacterioferritin (Rv3841)? Just Rv2384?
   1. Is there one gene that gives good accuracy
1. compare accuracy with ranom selection of genes
1. Classifier coefficient profile.
   1. Eliza genes - 1-10
   1. Auto selected genes: 5, 10, 20

# Preliminaries

## Imports

In [1]:
import init
from common import constants as cn
from common_python.util import dataframe
from tools import make_classification_data

import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import cross_val_score
import seaborn as sns

%matplotlib inline

In [2]:
data = make_classification_data.ClassificationData()
data.get(globals())

## Constants

In [3]:
FULL_RUN = True

STAGES = list(cn.STATE_NAMES)
STAGES.remove("Normoxia")

## Helper Functions

In [4]:
def plotSampleData(data_dct, genes= None, is_plot=True):
    """
    Plots trinary values of data for the specified genes.
    
    Parameters
    ----------
    data_dct: dict
        key: reference type
        value: sample dictionary
            key: sample name
            value: features
    genes: list-str
    """
    ref_types = list(data_dct.keys())
    sample_names = list(data_dct[ref_types[0]].keys())
    for sample_name in sample_names:
        for ref_type in ref_types:
            df_X = data_dct[ref_type][sample_name]
            if genes is None:
                genes = list(df_X.columns)
            TRINARY.plotExpressionLevels(genes, df_X=df_X, title=ref_type + ", " + sample_name, is_plot=is_plot)
            
# Tests
plotSampleData(SAMPLE_DCT, genes=MYCOBACTIN_BACTERIOFERRIN_GENES, is_plot=False)
print("OK!")

AttributeError: 'SampleData' object has no attribute 'keys'

In [ ]:
def reportSamplePredictions(data_dct):
    for name, df_X in data_dct.items():
        print("\n%s" % name)
        df_pred = CLASSIFIER.predict(df_X)
        new_columns = [STAGE_NAMES[c] for c in df_pred.columns]
        df_pred.columns = new_columns
        print(df_pred)

## Gene Expression

**Observations**
1. Prefer control values that avoid having a constant value of the trinary value over the samples (since such a situation is unhelpful for distinguishing classes).
1. On training data (Bioreactor)
   1. Many genes have the same within class values
   1. Few genes have the same between class values

**TO DO**: Plot data in pairs, 1 column, 2 rows; this allows comparisons of the expressions for T0 vs. pooled.

In [ ]:
# With Bioreactor reference data
plotSampleData(SAMPLE_DCT, genes=MYCOBACTIN_BACTERIOFERRIN_GENES)

In [ ]:
SAMPLE_DCT["ref_type_pooled"]["AW"]

# Condition Analysis

Analyzes data sets with distinct conditions.

In [ ]:
def plotConditions(sample_name, conditionFunc, fontsize_label=9, is_plot=True):
    """
    Plots conditions for all the expression reference calculations and classifiers.
    
    Parameters
    ----------
    sample_name: str
    conditionFunc: Function
    nrow: int
    ncol: int
    """
    ref_types = [REF_TYPE_BIOREACTOR, REF_TYPE_POOLED]
    ref_type_label_dct = {REF_TYPE_BIOREACTOR: "bioreactor T0",
                      REF_TYPE_POOLED: "pooled"}
    ncol = len(CLASSIFIER_DCT.keys())
    nrow = len(ref_types)
    _, axes = plt.subplots(nrow, ncol, figsize=(20, 10))
    icol = 0
    irow = 0
    for feature_ref_type in ref_types:
        for clf_key, clf in CLASSIFIER_DCT.items():
            ax = axes[irow, icol]
            df_X = SAMPLE_DCT[feature_ref_type][sample_name]
            clf.plotConditions(df_X, conditionFunc, state_names=STAGE_NAMES, ax=ax, is_plot=False,
                               fontsize_label=fontsize_label)
            if icol == 0:
                row_title = ref_type_label_dct[feature_ref_type]
                ax.set_ylabel(row_title, fontsize=16)
                # ax.text(-2, 5.5, row_title, fontsize=16)
            else:
                ax.set_yticklabels([])
            if irow == 0:
                ax.set_xticklabels([])
                plot_title = ", ".join(clf_key)
            else:
                plot_title = ""
            ax.set_title(plot_title, rotation=45)
            icol += 1
            if icol >= ncol:
                icol = 0
                irow += 1
    plt.suptitle(sample_name, fontsize=18)
    if not is_plot:
        plt.close()
    
# Tests
conditionFunc = lambda v: v[0:3].replace("_", "")
plotConditions("AM_MDM", conditionFunc, is_plot=False)

## AW

In [ ]:
conditionFunc = lambda v: v[0:-2]
plotConditions("AW", conditionFunc)

## AM, MDM

In [ ]:
SAMPLE_DCT[REF_TYPE_POOLED]["AM_MDM"]

In [ ]:
conditionFunc = lambda v: v[0:3].replace("_", "")
plotConditions("AM_MDM", conditionFunc)

## AW

In [ ]:
SAMPLE_DCT["ref_type_pooled"]["AW"].head(3)

## AM, MDM

In [ ]:
clf = CLASSIFIER_DCT[(T0, "all")]
df_X = SAMPLE_DCT[REF_TYPE_BIOREACTOR]["AM_MDM"]
conditionFunc = lambda v: v[0:3].replace("_", "")
_, ax = plt.subplots(1)
clf.plotConditions(df_X, conditionFunc, state_names=STAGE_NAMES, ax=ax, is_plot=False)
ax.set_title("AM, MDM")
plt.show()

In [ ]:
clf = CLASSIFIER_DCT[(POOLED, "all")]
sample_name = "GSE167232"
df_X = SAMPLE_DCT[REF_TYPE_POOLED][sample_name]
conditionFunc = lambda v: v[0:3].replace("_", "")
_, ax = plt.subplots(1)
clf.plotConditions(df_X, conditionFunc, state_names=STAGE_NAMES, ax=ax, is_plot=False)
ax.set_title(sample_name)
plt.show()

## GSE167232

In [ ]:
conditionFunc = lambda v: v[0:-2]
plotConditions("GSE167232", conditionFunc, fontsize_label=7)

# Progression Analysis

In [ ]:
#CLASSIFIER.plotFeatureContributions(df_X.loc["TB_AM1",:], class_names=STAGE_NAMES)
if FULL_RUN:
    df_predict = CLASSIFIER.predict(SAMPLE_DATA_DCT["GSE167232"])
    indexArr = np.reshape(list(df_predict.index), (4,3))
    num_row = 4
    num_col =3
    _, axes = plt.subplots(num_row, num_col, figsize=(20, 12))
    for irow in range(4):
        for icol in range(3):
            if icol == 0:
                is_ylabel = True
            else:
                is_ylabel = False
            if icol == num_col - 1:
                is_legend = True
            else:
                is_legend = False
            if irow == num_row - 1:
                is_xlabel = True
            else:
                is_xlabel = False
            instance = indexArr[irow, icol]
            ser_X = SAMPLE_DATA_DCT["GSE167232"].loc[instance, :]
            CLASSIFIER.plotFeatureContributions(ser_X, class_names=STAGE_NAMES, title=instance, ax=axes[irow,icol], is_plot=False,
                                               is_xlabel=is_xlabel, is_ylabel=is_ylabel, is_legend=is_legend)
    plt.show()

## Rustad

In [ ]:
x_vals = ['4h', '8h', '12h', '1da', '4da', '7da']
y_vals = [5, 4, 4, 3, 1, 4]
plt.plot(x_vals, y_vals)
x_vals = ['4h', '8h', '12h', '4da', '7da'] 
y_vals = [1, 3, 3, 2, 2]
plt.plot(x_vals, y_vals)

In [ ]:
def calcRustadParameters(feature_ref_type, clf_key):
    """
    Constructs the plot characteristics
    
    Parameters
    ----------
    feature_ref_type: str
    clf_key: (str, str)
    
    Returns
    -------
    df_X: DataFrame (feature vector)
    replFunc: Function (constructs replication strings)
    timeFunc: Function (constructs time strings)
    row_title: str (title for a row of plots)
    plot_title: str (plot title)
    """
    df_X = SAMPLE_DCT[feature_ref_type]["rustad"]
    sel =  ["H37Rv_hypoxia" in i and (not "10" in i) for i in df_X.index]
    df_X = df_X[sel]
    # Select just replication 1-3
    sel =  [any([r in i for r in ["rep1", "rep2", "rep3"]])  for i in df_X.index]
    replFunc = lambda i: i[-4:]
    timeFunc = lambda i: i[-9:-6].replace("_", "")
    plot_title = "--".join(clf_key)
    row_title = "T0" if feature_ref_type == make_classification_data.REF_TYPE_BIOREACTOR else "pooled"
    return df_X[sel], replFunc, timeFunc, row_title, plot_title

# Tests
df_X, replFunc, timeFunc, row_title, plot_title = calcRustadParameters("ref_type_bioreactor", ("T0", "all"))
assert(isinstance(df_X, pd.DataFrame))
assert(isinstance(plot_title, str))

In [ ]:
def plotProgression(func):
    """
    Does line plots for the sample.
    The plot row indicates how data are prepared. The columns indicate how
    the classifier is construted.
    
    Paramters
    ---------
    func: Function
        parameters: ref_type (str)
        returns: df_X, replFunc, timeFunc, title
    """
    ref_types = [REF_TYPE_BIOREACTOR, REF_TYPE_POOLED]
    ncol = len(CLASSIFIER_DCT.keys())
    nrow = len(ref_types)
    _, axes = plt.subplots(nrow, ncol, figsize=(12, 10))
    icol = 0
    irow = 0
    for feature_ref_type in ref_types:
        for clf_key, clf in CLASSIFIER_DCT.items():
            ax = axes[irow, icol]
            df_X, replFunc, timeFunc, row_title, plot_title = func(feature_ref_type, clf_key)
            clf.plotProgression(df_X, replFunc, timeFunc, ax=ax, is_plot=False)
            if icol == 0:
                ax.text(-2, 5.5, row_title, fontsize=16)
            else:
                ax.set_yticklabels([])
            if irow == 0:
                ax.set_xticklabels([])
            else:
                plot_title = ""
            ax.set_title(plot_title, rotation=45)
            icol += 1
            if icol >= ncol:
                icol = 0
                irow += 1
    plt.show()
            
plotProgression(calcRustadParameters)

## Galagan

In [ ]:
for ref_type in [REF_TYPE_BIOREACTOR, REF_TYPE_POOLED]:
    df_X = SAMPLE_DCT[ref_type]["galagan"]
    prediction_df = CLASSIFIER.predict(df_X)
    replFunc = lambda i: i[-4:]
    timeFunc = lambda i: i[0:2]
    title = "galagan: " + ref_type
    _, ax = plt.subplots(1)
    CLASSIFIER.plotProgression(df_X, replFunc, timeFunc, title=title, ax=ax)

# Summary Comments

1. Choice of control
   1. Criteria for choice
1. Evaluating the quality of classification for test samples
   1. Consistency between replications
   1. Consistency in stage progression of replication
1. Detailed summaries for data sets
1. Selection of genes based on classification accuracy selects some, but not all of the mycobactin genes. 
   1. Which are not selected?
   1. What additional genes are selected?
   1. What are the signs and magnitudes of the mycobactin genes in the "all" case?
1. Using pooled data results in very different gene selections